## Datasets: Lenguaje de consulta DataFrame vs Lambdas

La API de Datasets ofrece la opción de utilizar el lenguaje de consultas como las transformaciones de RDD

In [0]:
%scala
case class Person(id: Integer, firstName: String, middleName: String, lastName: String, gender: String, birthDate: String, ssn: String, salary: String)

val personDs = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter", ":").csv("dbfs:/databricks-datasets/learning-spark-v2/people/people-with-header-10m.txt").as[Person]

personDs.cache().count

defined class Person
personDs: org.apache.spark.sql.Dataset[Person] = [id: int, firstName: string ... 6 more fields]
res11: Long = 10000000

Consulta DataFrame DSL

In [0]:
%scala
println(personDs.filter($"firstName" === "Nell").distinct().count)

1218

Dataset con lambda

In [0]:
%scala
println(personDs.filter(x => x.firstName == "Nell").distinct().count)

1218

- Ventaja del uso de lambdas
  - Bueno para datos semiestructurados
  - Muy potente
- Desventajas
  - Catalyst no puede interpretar las lambdas hasta el tiempo de ejecución. 
  - Las lambdas son opacas para Catalyst. Como no sabe lo que está haciendo una lambda, no puede trasladarla a otra parte del procesamiento.
  - Saltar entre las lambdas y la API de consulta de DataFrame puede perjudicar el rendimiento.
  - Trabajar con lambdas significa que tenemos que `deserializar` del formato de Tungsteno a un objeto y luego volver a serializar al formato de Tungsteno cuando la lambda haya terminado.
  
Si tienes que usar lambdas, encadenarlas puede ayudar.

In [0]:
%scala
import java.util.Calendar

val earliestYear = Calendar.getInstance.get(Calendar.YEAR) - 40

personDs.filter(x => x.birthDate.split("-")(0).toInt > earliestYear)
        .filter($"salary" > 80000)
        .filter(x => x.lastName.startsWith("J"))
        .filter($"firstName".startsWith("D")).count()

import java.util.Calendar
earliestYear: Int = 1982
res15: Long = 1896

In [0]:
%scala
import org.apache.spark.sql.functions._

personDs.filter(year($"birthDate") > earliestYear)
        .filter($"salary" > 80000)
        .filter($"lastName".startsWith("J"))
        .filter($"firstName".startsWith("D")).count()

import org.apache.spark.sql.functions._
res16: Long = 1896